In [72]:
import sys
import sklearn
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import csv

In [64]:
# Helper functions
def convert_si_to_number(x):
    total_stars = 0
    if 'k' in x:
        if len(x) > 1:
            total_stars = float(x.replace('k', '')) * 1000 # convert k to a thousand
    elif 'M' in x:
        if len(x) > 1:
            total_stars = float(x.replace('M', '')) * 1000000 # convert M to a million
    elif 'B' in x:
        total_stars = float(x.replace('B', '')) * 1000000000 # convert B to a Billion
    else:
        total_stars = int(x) # Less than 1000
    
    return int(total_stars)

In [69]:
# Read file
filename = "30000-40000.csv"
file = open("../data/30000-40000.csv")
reader = csv.reader(file)
i = 0
y = []
X = []
names = []
temp = next(reader) # skip label row
for row in reader:
    if len(row) < 6: # exclude data missing fields
        continue
    name = row[0]
    location = row[1]
    score = row[2]
    reviews = row[3]
    salaries = row[4]
    interviews = row[5]
    if len(location.split(",")) < 2: # exclude data missing states or cities
        continue
    
    names.append(name)
        
    # X features
    feature = []
    city = location.split(",")[0]
    state = location.split(",")[1]
    feature.append(city)
    feature.append(state)
    feature.append(convert_si_to_number(reviews))
    if salaries == '--':
        feature.append(0)
    else:
        feature.append(convert_si_to_number(salaries))
    if interviews == '--':
        feature.append(0)
    else:
        feature.append(convert_si_to_number(interviews))
    X.append(feature)

    # y labels
    y.append(score)
    
    i = i + 1

In [71]:
state

' AZ'

In [66]:
# Convert to numpy array
X = np.array(X).astype(np.float)
y = np.array(y).astype(np.float)

In [67]:
# Train Model
model = LinearRegression()
model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [68]:
# Make predictions
predictions = model.predict(X)

In [60]:
# Calculate statistics
error = mean_squared_error(y, predictions)
w = model.coef_
b = model.intercept_

In [74]:
enc = OneHotEncoder(handle_unknown='ignore')
X = [['Male', 1], ['Female', 3], ['Female', 2]]
enc.fit(X)

ValueError: could not convert string to float: 'Male'